# Analysis of Load Divergence by State

This notebook analyzes the time-series of the annual mean difference in load projections by state across pairs of IM3 scenarios.

In [1]:
# Start by importing the packages we need:
import os

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd

from glob import glob
from mpl_toolkits.axes_grid1 import make_axes_locatable


## Set the Directory Structure

In [2]:
# Identify the top-level directory and the subdirectory where the data will be stored:
data_input_dir =  '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/outputs/tell_output/'
data_output_dir =  '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2023_tbd/data/divergence/'
image_output_dir =  '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2023_tbd/figures/States/'


## Process the Annual Mean Load Difference

In [3]:
# Define a function to calculate the annual mean load difference between 'scenario_one' and 'scenario_two':
def process_state_load_difference_time_series(data_input_dir: str, scenario_one: str, scenario_two: str):
    # Initiate a counter to store the results:
    counter = 0;
    output_df = pd.DataFrame()
    
    # Loop over the years from 2020 to 2095 in 5 year increments:
    for year in range(2020,2099,5):
        # Read in the state-level time series for 'scenario_one':
        df_one = pd.read_csv(data_input_dir + scenario_one + '/' + str(year) + '/TELL_State_Hourly_Load_Data_' + str(year) + '_Scaled_' + str(year) + '.csv')
            
        # Rename a few columns for simplicity:
        df_one.rename(columns={'Scaled_TELL_State_Load_MWh': 'Load_One_MWh'}, inplace=True)
        
        # Only keep the columns that are needed:
        df_one = df_one[['State_Name', 'Time_UTC', 'Load_One_MWh']].copy()
        
        # Read in the state-level time series for 'scenario_two':
        df_two = pd.read_csv(data_input_dir + scenario_two + '/' + str(year) + '/TELL_State_Hourly_Load_Data_' + str(year) + '_Scaled_' + str(year) + '.csv')
            
        # Rename a few columns for simplicity:
        df_two.rename(columns={'Scaled_TELL_State_Load_MWh': 'Load_Two_MWh'}, inplace=True)
        
        # Only keep the columns that are needed:
        df_two = df_two[['State_Name', 'Time_UTC', 'Load_Two_MWh']].copy()
         
        # Merge the two time series together on state name and time:
        merged_df = df_one.merge(df_two, on=['State_Name', 'Time_UTC'])
            
        # Calculate the mean load between the two time-series as well as the absolute and relative load biases:
        merged_df['Mean_Load_MWh'] =  merged_df[['Load_One_MWh', 'Load_Two_MWh']].mean(axis=1)
        merged_df['Absolute_Difference_MWh'] = abs(merged_df['Load_One_MWh'] - merged_df['Load_Two_MWh'])
        merged_df['Relative_Difference_%'] = 100*(merged_df['Absolute_Difference_MWh']/merged_df['Mean_Load_MWh']).round(4)
       
        # Make a list of all of the states in the 'merged_df':
        states = merged_df['State_Name'].unique()
         
        # Loop over the states and calculate the mean bias for each state:
        for i in range(len(states)):
            # Iterate the counter by one:
            counter = counter + 1
            
            # Subset to just the data for the state being processed:
            subset_df = merged_df[merged_df['State_Name'].isin([states[i]])].copy()
          
            # Subset to the top 10% of all mean load hours:
            peak_df = subset_df.nlargest(876,'Mean_Load_MWh')
        
            # Put the output in a new dataframe:
            output_df.loc[counter, 'Year'] = str(year)
            output_df.loc[counter, 'State'] = states[i]
            output_df.loc[counter, 'Mean_Abs_Diff_MWh'] = subset_df['Absolute_Difference_MWh'].mean().round(2)
            output_df.loc[counter, 'Mean_Rel_Diff_%'] = subset_df['Relative_Difference_%'].mean().round(2)
            output_df.loc[counter, 'Peak_Abs_Diff_MWh'] = peak_df['Absolute_Difference_MWh'].mean().round(2)
            output_df.loc[counter, 'Peak_Rel_Diff_%'] = peak_df['Relative_Difference_%'].mean().round(2)
        
            # Clean up and move to the next step in the loop:
            del subset_df, peak_df
    
    # Generate the .csv output file name:
    csv_output_filename = os.path.join(data_output_dir, 'State_Differences_' + scenario_one + '_Versus_' + scenario_two + '.csv')
    
    # Write out the dataframe to a .csv file:
    output_df.to_csv(csv_output_filename, sep=',', index=False)
    
    return output_df


In [4]:
# Set the groups of scenarios to plot:
ssp_div_a_one = 'rcp45cooler_ssp5'; ssp_div_a_two = 'rcp45cooler_ssp3';
ssp_div_b_one = 'rcp45hotter_ssp5'; ssp_div_b_two = 'rcp45hotter_ssp3';
ssp_div_c_one = 'rcp85cooler_ssp5'; ssp_div_c_two = 'rcp85cooler_ssp3';
ssp_div_d_one = 'rcp85hotter_ssp5'; ssp_div_d_two = 'rcp85hotter_ssp3';
  
rcp_div_a_one = 'rcp85cooler_ssp3'; rcp_div_a_two = 'rcp45cooler_ssp3'; 
rcp_div_b_one = 'rcp85hotter_ssp3'; rcp_div_b_two = 'rcp45hotter_ssp3'; 
rcp_div_c_one = 'rcp85cooler_ssp5'; rcp_div_c_two = 'rcp45cooler_ssp5'; 
rcp_div_d_one = 'rcp85hotter_ssp5'; rcp_div_d_two = 'rcp45hotter_ssp5';
    
gcm_div_a_one = 'rcp45hotter_ssp3'; gcm_div_a_two = 'rcp45cooler_ssp3'; 
gcm_div_b_one = 'rcp45hotter_ssp5'; gcm_div_b_two = 'rcp45cooler_ssp5'; 
gcm_div_c_one = 'rcp85hotter_ssp3'; gcm_div_c_two = 'rcp85cooler_ssp3'; 
gcm_div_d_one = 'rcp85hotter_ssp5'; gcm_div_d_two = 'rcp85cooler_ssp5';

# Process the load time series for each pair of scenarios
output_df = process_state_load_difference_time_series(data_input_dir = data_input_dir, 
                                                      scenario_one = ssp_div_a_one, 
                                                      scenario_two = ssp_div_a_two)

output_df


,Year,State,Mean_Abs_Diff_MWh,Mean_Rel_Diff_%,Peak_Abs_Diff_MWh,Peak_Rel_Diff_%
1,2020,Alabama,67.73,0.65,107.07,0.69
2,2020,Arizona,118.21,1.30,53.75,0.31
3,2020,Arkansas,8.10,0.15,10.04,0.13
4,2020,California,1039.28,2.93,1408.47,2.70
5,2020,Colorado,109.22,1.57,124.13,1.32
...,...,...,...,...,...,...
780,2095,Virginia,17635.94,84.52,25941.27,84.97
781,2095,Washington,14631.98,67.65,18178.47,67.56
782,2095,West Virginia,4359.53,83.32,6412.01,83.73
783,2095,Wisconsin,9896.11,64.38,13479.23,64.46


## Make the Plots

In [5]:
# Define a function to plot the time series for a single state:
def plot_state_load_difference_time_series(state_to_plot: str, data_input_dir: str, image_output_dir: str, image_resolution: int, save_images=False):
    
    # Set the groups of scenarios to plot:
    ssp_div_a_one = 'rcp45cooler_ssp5'; ssp_div_a_two = 'rcp45cooler_ssp3';
    ssp_div_b_one = 'rcp45hotter_ssp5'; ssp_div_b_two = 'rcp45hotter_ssp3';
    ssp_div_c_one = 'rcp85cooler_ssp5'; ssp_div_c_two = 'rcp85cooler_ssp3';
    ssp_div_d_one = 'rcp85hotter_ssp5'; ssp_div_d_two = 'rcp85hotter_ssp3';
    rcp_div_a_one = 'rcp85cooler_ssp3'; rcp_div_a_two = 'rcp45cooler_ssp3'; 
    rcp_div_b_one = 'rcp85hotter_ssp3'; rcp_div_b_two = 'rcp45hotter_ssp3'; 
    rcp_div_c_one = 'rcp85cooler_ssp5'; rcp_div_c_two = 'rcp45cooler_ssp5'; 
    rcp_div_d_one = 'rcp85hotter_ssp5'; rcp_div_d_two = 'rcp45hotter_ssp5';
    gcm_div_a_one = 'rcp45hotter_ssp3'; gcm_div_a_two = 'rcp45cooler_ssp3'; 
    gcm_div_b_one = 'rcp45hotter_ssp5'; gcm_div_b_two = 'rcp45cooler_ssp5'; 
    gcm_div_c_one = 'rcp85hotter_ssp3'; gcm_div_c_two = 'rcp85cooler_ssp3'; 
    gcm_div_d_one = 'rcp85hotter_ssp5'; gcm_div_d_two = 'rcp85cooler_ssp5';
    
    # Read in the output created by the function above:
    ssp_div_a_df = pd.read_csv(data_input_dir + 'State_Differences_' + ssp_div_a_one + '_Versus_' + ssp_div_a_two + '.csv'); ssp_div_a_df = ssp_div_a_df[ssp_div_a_df['State'].isin([state_to_plot])]
    ssp_div_b_df = pd.read_csv(data_input_dir + 'State_Differences_' + ssp_div_b_one + '_Versus_' + ssp_div_b_two + '.csv'); ssp_div_b_df = ssp_div_b_df[ssp_div_b_df['State'].isin([state_to_plot])]
    ssp_div_c_df = pd.read_csv(data_input_dir + 'State_Differences_' + ssp_div_c_one + '_Versus_' + ssp_div_c_two + '.csv'); ssp_div_c_df = ssp_div_c_df[ssp_div_c_df['State'].isin([state_to_plot])]
    ssp_div_d_df = pd.read_csv(data_input_dir + 'State_Differences_' + ssp_div_d_one + '_Versus_' + ssp_div_d_two + '.csv'); ssp_div_d_df = ssp_div_d_df[ssp_div_d_df['State'].isin([state_to_plot])] 
        
    rcp_div_a_df = pd.read_csv(data_input_dir + 'State_Differences_' + rcp_div_a_one + '_Versus_' + rcp_div_a_two + '.csv'); rcp_div_a_df = rcp_div_a_df[rcp_div_a_df['State'].isin([state_to_plot])]
    rcp_div_b_df = pd.read_csv(data_input_dir + 'State_Differences_' + rcp_div_b_one + '_Versus_' + rcp_div_b_two + '.csv'); rcp_div_b_df = rcp_div_b_df[rcp_div_b_df['State'].isin([state_to_plot])]
    rcp_div_c_df = pd.read_csv(data_input_dir + 'State_Differences_' + rcp_div_c_one + '_Versus_' + rcp_div_c_two + '.csv'); rcp_div_c_df = rcp_div_c_df[rcp_div_c_df['State'].isin([state_to_plot])]
    rcp_div_d_df = pd.read_csv(data_input_dir + 'State_Differences_' + rcp_div_d_one + '_Versus_' + rcp_div_d_two + '.csv'); rcp_div_d_df = rcp_div_d_df[rcp_div_d_df['State'].isin([state_to_plot])] 
    
    gcm_div_a_df = pd.read_csv(data_input_dir + 'State_Differences_' + gcm_div_a_one + '_Versus_' + gcm_div_a_two + '.csv'); gcm_div_a_df = gcm_div_a_df[gcm_div_a_df['State'].isin([state_to_plot])]
    gcm_div_b_df = pd.read_csv(data_input_dir + 'State_Differences_' + gcm_div_b_one + '_Versus_' + gcm_div_b_two + '.csv'); gcm_div_b_df = gcm_div_b_df[gcm_div_b_df['State'].isin([state_to_plot])]
    gcm_div_c_df = pd.read_csv(data_input_dir + 'State_Differences_' + gcm_div_c_one + '_Versus_' + gcm_div_c_two + '.csv'); gcm_div_c_df = gcm_div_c_df[gcm_div_c_df['State'].isin([state_to_plot])]
    gcm_div_d_df = pd.read_csv(data_input_dir + 'State_Differences_' + gcm_div_d_one + '_Versus_' + gcm_div_d_two + '.csv'); gcm_div_d_df = gcm_div_d_df[gcm_div_d_df['State'].isin([state_to_plot])] 
    

    # Make the plot:
    plt.figure(figsize=(24, 15))
    plt.rcParams['font.size'] = 16
    
    plt.subplot(221)
    plt.plot(-5, 0, color='k', linestyle='-', label='Mean Load', linewidth=2)
    plt.plot(-5, 0, color='k', linestyle=':', label='Peak Load', linewidth=2)
    plt.plot(ssp_div_a_df['Year'], ssp_div_a_df['Mean_Rel_Diff_%'], color='b', linestyle='-', label=(ssp_div_a_one + ' vs ' + ssp_div_a_two), linewidth=2)
    plt.plot(ssp_div_b_df['Year'], ssp_div_b_df['Mean_Rel_Diff_%'], color='cyan', linestyle='-', label=(ssp_div_b_one + ' vs ' + ssp_div_b_two), linewidth=2)
    plt.plot(ssp_div_c_df['Year'], ssp_div_c_df['Mean_Rel_Diff_%'], color='orange', linestyle='-', label=(ssp_div_c_one + ' vs ' + ssp_div_c_two), linewidth=2)
    plt.plot(ssp_div_d_df['Year'], ssp_div_d_df['Mean_Rel_Diff_%'], color='r', linestyle='-', label=(ssp_div_d_one + ' vs ' + ssp_div_d_two), linewidth=2)
    plt.plot(ssp_div_a_df['Year'], ssp_div_a_df['Peak_Rel_Diff_%'], color='b', linestyle=':', linewidth=2)
    plt.plot(ssp_div_b_df['Year'], ssp_div_b_df['Peak_Rel_Diff_%'], color='cyan', linestyle=':', linewidth=2)
    plt.plot(ssp_div_c_df['Year'], ssp_div_c_df['Peak_Rel_Diff_%'], color='orange', linestyle=':', linewidth=2)
    plt.plot(ssp_div_d_df['Year'], ssp_div_d_df['Peak_Rel_Diff_%'], color='r', linestyle=':', linewidth=2)
    plt.xlim([2020, 2100]); plt.ylim(bottom=0)
    plt.grid(True)
    plt.legend(loc='upper left', prop={'size': 14})
    plt.xlabel('Year'); plt.ylabel('Annual Mean Hourly Load Difference [%]')
    plt.title((state_to_plot + ' Socioeconomic Scenario Divergence [SSP3 vs SSP5]'))
    
    plt.subplot(222)
    plt.plot(-5, 0, color='k', linestyle='-', label='Mean Load', linewidth=2)
    plt.plot(-5, 0, color='k', linestyle=':', label='Peak Load', linewidth=2)
    plt.plot(rcp_div_a_df['Year'], rcp_div_a_df['Mean_Rel_Diff_%'], color='forestgreen', linestyle='-', label=(rcp_div_a_one + ' vs ' + rcp_div_a_two), linewidth=2)
    plt.plot(rcp_div_b_df['Year'], rcp_div_b_df['Mean_Rel_Diff_%'], color='blueviolet', linestyle='-', label=(rcp_div_b_one + ' vs ' + rcp_div_b_two), linewidth=2)
    plt.plot(rcp_div_c_df['Year'], rcp_div_c_df['Mean_Rel_Diff_%'], color='goldenrod', linestyle='-', label=(rcp_div_c_one + ' vs ' + rcp_div_c_two), linewidth=2)
    plt.plot(rcp_div_d_df['Year'], rcp_div_d_df['Mean_Rel_Diff_%'], color='lightcoral', linestyle='-', label=(rcp_div_d_one + ' vs ' + rcp_div_d_two), linewidth=2)
    plt.plot(rcp_div_a_df['Year'], rcp_div_a_df['Peak_Rel_Diff_%'], color='forestgreen', linestyle=':', linewidth=2)
    plt.plot(rcp_div_b_df['Year'], rcp_div_b_df['Peak_Rel_Diff_%'], color='blueviolet', linestyle=':', linewidth=2)
    plt.plot(rcp_div_c_df['Year'], rcp_div_c_df['Peak_Rel_Diff_%'], color='goldenrod', linestyle=':', linewidth=2)
    plt.plot(rcp_div_d_df['Year'], rcp_div_d_df['Peak_Rel_Diff_%'], color='lightcoral', linestyle=':', linewidth=2)
    plt.xlim([2020, 2100]); plt.ylim(bottom=0)
    plt.grid(True)
    plt.legend(loc='upper left', prop={'size': 14})
    plt.xlabel('Year'); plt.ylabel('Annual Mean Hourly Load Difference [%]')
    plt.title((state_to_plot + ' Climate Scenario Divergence [RCP 4.5 vs RCP 8.5]'))
        
    plt.subplot(223)
    plt.plot(-5, 0, color='k', linestyle='-', label='Mean Load', linewidth=2)
    plt.plot(-5, 0, color='k', linestyle=':', label='Peak Load', linewidth=2)
    plt.plot(gcm_div_a_df['Year'], gcm_div_a_df['Mean_Rel_Diff_%'], color='brown', linestyle='-', label=(gcm_div_a_one + ' vs ' + gcm_div_a_two), linewidth=2)
    plt.plot(gcm_div_b_df['Year'], gcm_div_b_df['Mean_Rel_Diff_%'], color='gold', linestyle='-', label=(gcm_div_b_one + ' vs ' + gcm_div_b_two), linewidth=2)
    plt.plot(gcm_div_c_df['Year'], gcm_div_c_df['Mean_Rel_Diff_%'], color='skyblue', linestyle='-', label=(gcm_div_c_one + ' vs ' + gcm_div_c_two), linewidth=2)
    plt.plot(gcm_div_d_df['Year'], gcm_div_d_df['Mean_Rel_Diff_%'], color='magenta', linestyle='-', label=(gcm_div_d_one + ' vs ' + gcm_div_d_two), linewidth=2)
    plt.plot(gcm_div_a_df['Year'], gcm_div_a_df['Peak_Rel_Diff_%'], color='brown', linestyle=':', linewidth=2)
    plt.plot(gcm_div_b_df['Year'], gcm_div_b_df['Peak_Rel_Diff_%'], color='gold', linestyle=':', linewidth=2)
    plt.plot(gcm_div_c_df['Year'], gcm_div_c_df['Peak_Rel_Diff_%'], color='skyblue', linestyle=':', linewidth=2)
    plt.plot(gcm_div_d_df['Year'], gcm_div_d_df['Peak_Rel_Diff_%'], color='magenta', linestyle=':', linewidth=2)
    plt.xlim([2020, 2100]); plt.ylim(bottom=0)
    plt.grid(True)
    plt.legend(loc='upper left', prop={'size': 14})
    plt.xlabel('Year'); plt.ylabel('Annual Mean Hourly Load Difference [%]')
    plt.title((state_to_plot + ' Climate Model Divergence [Hotter vs Colder Models]'))
    
    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       filename = ('State_Load_Difference_' + state_to_plot.replace(" ", "_") + '.png')
       plt.savefig(os.path.join(image_output_dir, filename), dpi=image_resolution, bbox_inches='tight', facecolor='white')
       plt.close()


In [6]:
# Make a list of all of the states in the 'output_df' created above:
states = output_df['State'].unique()
         
# Loop over the states and make the plot for each state:
for i in range(len(states)):
    plot_state_load_difference_time_series(state_to_plot = states[i],
                                           data_input_dir = data_output_dir, 
                                           image_output_dir = image_output_dir, 
                                           image_resolution = 50, 
                                           save_images = True)


In [11]:
# Define a function to plot the distribution of load differences across states:
def plot_state_difference_box_plots(data_input_dir: str, image_output_dir: str, image_resolution: int, save_images=False):
    
    # Set the groups of scenarios to plot:
    ssp_div_a_one = 'rcp45cooler_ssp5'; ssp_div_a_two = 'rcp45cooler_ssp3';
    ssp_div_b_one = 'rcp45hotter_ssp5'; ssp_div_b_two = 'rcp45hotter_ssp3';
    ssp_div_c_one = 'rcp85cooler_ssp5'; ssp_div_c_two = 'rcp85cooler_ssp3';
    ssp_div_d_one = 'rcp85hotter_ssp5'; ssp_div_d_two = 'rcp85hotter_ssp3';
    rcp_div_a_one = 'rcp85cooler_ssp3'; rcp_div_a_two = 'rcp45cooler_ssp3'; 
    rcp_div_b_one = 'rcp85hotter_ssp3'; rcp_div_b_two = 'rcp45hotter_ssp3'; 
    rcp_div_c_one = 'rcp85cooler_ssp5'; rcp_div_c_two = 'rcp45cooler_ssp5'; 
    rcp_div_d_one = 'rcp85hotter_ssp5'; rcp_div_d_two = 'rcp45hotter_ssp5';
    gcm_div_a_one = 'rcp45hotter_ssp3'; gcm_div_a_two = 'rcp45cooler_ssp3'; 
    gcm_div_b_one = 'rcp45hotter_ssp5'; gcm_div_b_two = 'rcp45cooler_ssp5'; 
    gcm_div_c_one = 'rcp85hotter_ssp3'; gcm_div_c_two = 'rcp85cooler_ssp3'; 
    gcm_div_d_one = 'rcp85hotter_ssp5'; gcm_div_d_two = 'rcp85cooler_ssp5';
    
    # Read in the output created by the function above:
    ssp_div_a_df = pd.read_csv(data_input_dir + 'State_Differences_' + ssp_div_a_one + '_Versus_' + ssp_div_a_two + '.csv');
    ssp_div_b_df = pd.read_csv(data_input_dir + 'State_Differences_' + ssp_div_b_one + '_Versus_' + ssp_div_b_two + '.csv');
    ssp_div_c_df = pd.read_csv(data_input_dir + 'State_Differences_' + ssp_div_c_one + '_Versus_' + ssp_div_c_two + '.csv');
    ssp_div_d_df = pd.read_csv(data_input_dir + 'State_Differences_' + ssp_div_d_one + '_Versus_' + ssp_div_d_two + '.csv');
        
    rcp_div_a_df = pd.read_csv(data_input_dir + 'State_Differences_' + rcp_div_a_one + '_Versus_' + rcp_div_a_two + '.csv');
    rcp_div_b_df = pd.read_csv(data_input_dir + 'State_Differences_' + rcp_div_b_one + '_Versus_' + rcp_div_b_two + '.csv');
    rcp_div_c_df = pd.read_csv(data_input_dir + 'State_Differences_' + rcp_div_c_one + '_Versus_' + rcp_div_c_two + '.csv');
    rcp_div_d_df = pd.read_csv(data_input_dir + 'State_Differences_' + rcp_div_d_one + '_Versus_' + rcp_div_d_two + '.csv'); 
    
    gcm_div_a_df = pd.read_csv(data_input_dir + 'State_Differences_' + gcm_div_a_one + '_Versus_' + gcm_div_a_two + '.csv');
    gcm_div_b_df = pd.read_csv(data_input_dir + 'State_Differences_' + gcm_div_b_one + '_Versus_' + gcm_div_b_two + '.csv');
    gcm_div_c_df = pd.read_csv(data_input_dir + 'State_Differences_' + gcm_div_c_one + '_Versus_' + gcm_div_c_two + '.csv');
    gcm_div_d_df = pd.read_csv(data_input_dir + 'State_Differences_' + gcm_div_d_one + '_Versus_' + gcm_div_d_two + '.csv');
    
    # Set the style parameters for the box plots:
    boxprops1a=dict(color='b',      linestyle='-',  linewidth=2); medianprops1a=dict(color='b',      linewidth=2); whiskerprops1a=dict(color='b',      linewidth=2, linestyle='-');  flierprops1a=dict(color='b',      markeredgecolor='b');      capprops1a=dict(color='b',      linewidth=0);
    boxprops2a=dict(color='b',      linestyle='--', linewidth=2); medianprops2a=dict(color='b',      linewidth=2); whiskerprops2a=dict(color='b',      linewidth=2, linestyle='--'); flierprops2a=dict(color='b',      markeredgecolor='b');      capprops2a=dict(color='b',      linewidth=0);
    boxprops3a=dict(color='cyan',   linestyle='-',  linewidth=2); medianprops3a=dict(color='cyan',   linewidth=2); whiskerprops3a=dict(color='cyan',   linewidth=2, linestyle='-');  flierprops3a=dict(color='cyan',   markeredgecolor='cyan');   capprops3a=dict(color='cyan',   linewidth=0);
    boxprops4a=dict(color='cyan',   linestyle='--', linewidth=2); medianprops4a=dict(color='cyan',   linewidth=2); whiskerprops4a=dict(color='cyan',   linewidth=2, linestyle='--'); flierprops4a=dict(color='cyan',   markeredgecolor='cyan');   capprops4a=dict(color='cyan',   linewidth=0);
    boxprops5a=dict(color='orange', linestyle='-',  linewidth=2); medianprops5a=dict(color='orange', linewidth=2); whiskerprops5a=dict(color='orange', linewidth=2, linestyle='-');  flierprops5a=dict(color='orange', markeredgecolor='orange'); capprops5a=dict(color='orange', linewidth=0);
    boxprops6a=dict(color='orange', linestyle='--', linewidth=2); medianprops6a=dict(color='orange', linewidth=2); whiskerprops6a=dict(color='orange', linewidth=2, linestyle='--'); flierprops6a=dict(color='orange', markeredgecolor='orange'); capprops6a=dict(color='orange', linewidth=0);
    boxprops7a=dict(color='r',      linestyle='-',  linewidth=2); medianprops7a=dict(color='r',      linewidth=2); whiskerprops7a=dict(color='r',      linewidth=2, linestyle='-');  flierprops7a=dict(color='r',      markeredgecolor='r');      capprops7a=dict(color='r',      linewidth=0);
    boxprops8a=dict(color='r',      linestyle='--', linewidth=2); medianprops8a=dict(color='r',      linewidth=2); whiskerprops8a=dict(color='r',      linewidth=2, linestyle='--'); flierprops8a=dict(color='r',      markeredgecolor='r');      capprops8a=dict(color='r',      linewidth=0);
    
    boxprops1b=dict(color='forestgreen', linestyle='-',  linewidth=2); medianprops1b=dict(color='forestgreen', linewidth=2); whiskerprops1b=dict(color='forestgreen', linewidth=2, linestyle='-');  flierprops1b=dict(color='forestgreen', markeredgecolor='forestgreen'); capprops1b=dict(color='forestgreen', linewidth=0);
    boxprops2b=dict(color='forestgreen', linestyle='--', linewidth=2); medianprops2b=dict(color='forestgreen', linewidth=2); whiskerprops2b=dict(color='forestgreen', linewidth=2, linestyle='--'); flierprops2b=dict(color='forestgreen', markeredgecolor='forestgreen'); capprops2b=dict(color='forestgreen', linewidth=0);
    boxprops3b=dict(color='blueviolet',  linestyle='-',  linewidth=2); medianprops3b=dict(color='blueviolet',  linewidth=2); whiskerprops3b=dict(color='blueviolet',  linewidth=2, linestyle='-');  flierprops3b=dict(color='blueviolet',  markeredgecolor='blueviolet');  capprops3b=dict(color='blueviolet',  linewidth=0);
    boxprops4b=dict(color='blueviolet',  linestyle='--', linewidth=2); medianprops4b=dict(color='blueviolet',  linewidth=2); whiskerprops4b=dict(color='blueviolet',  linewidth=2, linestyle='--'); flierprops4b=dict(color='blueviolet',  markeredgecolor='blueviolet');  capprops4b=dict(color='blueviolet',  linewidth=0);
    boxprops5b=dict(color='goldenrod',   linestyle='-',  linewidth=2); medianprops5b=dict(color='goldenrod',   linewidth=2); whiskerprops5b=dict(color='goldenrod',   linewidth=2, linestyle='-');  flierprops5b=dict(color='goldenrod',   markeredgecolor='goldenrod');   capprops5b=dict(color='goldenrod',   linewidth=0);
    boxprops6b=dict(color='goldenrod',   linestyle='--', linewidth=2); medianprops6b=dict(color='goldenrod',   linewidth=2); whiskerprops6b=dict(color='goldenrod',   linewidth=2, linestyle='--'); flierprops6b=dict(color='goldenrod',   markeredgecolor='goldenrod');   capprops6b=dict(color='goldenrod',   linewidth=0);
    boxprops7b=dict(color='lightcoral',  linestyle='-',  linewidth=2); medianprops7b=dict(color='lightcoral',  linewidth=2); whiskerprops7b=dict(color='lightcoral',  linewidth=2, linestyle='-');  flierprops7b=dict(color='lightcoral',  markeredgecolor='lightcoral');  capprops7b=dict(color='lightcoral',  linewidth=0);
    boxprops8b=dict(color='lightcoral',  linestyle='--', linewidth=2); medianprops8b=dict(color='lightcoral',  linewidth=2); whiskerprops8b=dict(color='lightcoral',  linewidth=2, linestyle='--'); flierprops8b=dict(color='lightcoral',  markeredgecolor='lightcoral');  capprops8b=dict(color='lightcoral',  linewidth=0);
    
    boxprops1c=dict(color='brown',   linestyle='-',  linewidth=2); medianprops1c=dict(color='brown',   linewidth=2); whiskerprops1c=dict(color='brown',   linewidth=2, linestyle='-');  flierprops1c=dict(color='brown',   markeredgecolor='brown');   capprops1c=dict(color='brown',   linewidth=0);
    boxprops2c=dict(color='brown',   linestyle='--', linewidth=2); medianprops2c=dict(color='brown',   linewidth=2); whiskerprops2c=dict(color='brown',   linewidth=2, linestyle='--'); flierprops2c=dict(color='brown',   markeredgecolor='brown');   capprops2c=dict(color='brown',   linewidth=0);
    boxprops3c=dict(color='gold',    linestyle='-',  linewidth=2); medianprops3c=dict(color='gold',    linewidth=2); whiskerprops3c=dict(color='gold',    linewidth=2, linestyle='-');  flierprops3c=dict(color='gold',    markeredgecolor='gold');    capprops3c=dict(color='gold',    linewidth=0);
    boxprops4c=dict(color='gold',    linestyle='--', linewidth=2); medianprops4c=dict(color='gold',    linewidth=2); whiskerprops4c=dict(color='gold',    linewidth=2, linestyle='--'); flierprops4c=dict(color='gold',    markeredgecolor='gold');    capprops4c=dict(color='gold',    linewidth=0);
    boxprops5c=dict(color='skyblue', linestyle='-',  linewidth=2); medianprops5c=dict(color='skyblue', linewidth=2); whiskerprops5c=dict(color='skyblue', linewidth=2, linestyle='-');  flierprops5c=dict(color='skyblue', markeredgecolor='skyblue'); capprops5c=dict(color='skyblue', linewidth=0);
    boxprops6c=dict(color='skyblue', linestyle='--', linewidth=2); medianprops6c=dict(color='skyblue', linewidth=2); whiskerprops6c=dict(color='skyblue', linewidth=2, linestyle='--'); flierprops6c=dict(color='skyblue', markeredgecolor='skyblue'); capprops6c=dict(color='skyblue', linewidth=0);
    boxprops7c=dict(color='magenta', linestyle='-',  linewidth=2); medianprops7c=dict(color='magenta', linewidth=2); whiskerprops7c=dict(color='magenta', linewidth=2, linestyle='-');  flierprops7c=dict(color='magenta', markeredgecolor='magenta'); capprops7c=dict(color='magenta', linewidth=0);
    boxprops8c=dict(color='magenta', linestyle='--', linewidth=2); medianprops8c=dict(color='magenta', linewidth=2); whiskerprops8c=dict(color='magenta', linewidth=2, linestyle='--'); flierprops8c=dict(color='magenta', markeredgecolor='magenta'); capprops8c=dict(color='magenta', linewidth=0);
    
    # Make the raw load plots:
    plt.figure(figsize=(24, 20))
    plt.rcParams['font.size'] = 14
    
    # Plot the distribution of the change in annual total load across states:
    plt.subplot(311)
    plt.plot([0,0], color='k',      linestyle='-', label='Mean Load', linewidth=2)
    plt.plot([0,0], color='k',      linestyle=':', label='Peak Load', linewidth=2)
    plt.plot([0,0], color='b',      linestyle='-', label=(ssp_div_a_one + ' vs ' + ssp_div_a_two), linewidth=2)
    plt.plot([0,0], color='cyan',   linestyle='-', label=(ssp_div_b_one + ' vs ' + ssp_div_b_two), linewidth=2)
    plt.plot([0,0], color='orange', linestyle='-', label=(ssp_div_c_one + ' vs ' + ssp_div_c_two), linewidth=2)
    plt.plot([0,0], color='r',      linestyle='-', label=(ssp_div_d_one + ' vs ' + ssp_div_d_two), linewidth=2)
    for year in range(2020,2100,10):
        plt.boxplot(ssp_div_a_df['Mean_Rel_Diff_%'].loc[(ssp_div_a_df['Year']==year)], vert=True, positions=[year-2.8], widths=0.6, 
                    medianprops=medianprops1a, whiskerprops=whiskerprops1a, flierprops=flierprops1a, boxprops=boxprops1a, capprops=capprops1a)
        plt.boxplot(ssp_div_a_df['Peak_Rel_Diff_%'].loc[(ssp_div_a_df['Year']==year)], vert=True, positions=[year-2.0], widths=0.6, 
                    medianprops=medianprops2a, whiskerprops=whiskerprops2a, flierprops=flierprops2a, boxprops=boxprops2a, capprops=capprops2a)
        plt.boxplot(ssp_div_b_df['Mean_Rel_Diff_%'].loc[(ssp_div_b_df['Year']==year)], vert=True, positions=[year-1.2], widths=0.6, 
                    medianprops=medianprops3a, whiskerprops=whiskerprops3a, flierprops=flierprops3a, boxprops=boxprops3a, capprops=capprops3a)
        plt.boxplot(ssp_div_b_df['Peak_Rel_Diff_%'].loc[(ssp_div_b_df['Year']==year)], vert=True, positions=[year-0.4], widths=0.6, 
                    medianprops=medianprops4a, whiskerprops=whiskerprops4a, flierprops=flierprops4a, boxprops=boxprops4a, capprops=capprops4a)
        plt.boxplot(ssp_div_c_df['Mean_Rel_Diff_%'].loc[(ssp_div_c_df['Year']==year)], vert=True, positions=[year+0.4], widths=0.6, 
                    medianprops=medianprops5a, whiskerprops=whiskerprops5a, flierprops=flierprops5a, boxprops=boxprops5a, capprops=capprops5a)
        plt.boxplot(ssp_div_c_df['Peak_Rel_Diff_%'].loc[(ssp_div_c_df['Year']==year)], vert=True, positions=[year+1.2], widths=0.6, 
                    medianprops=medianprops6a, whiskerprops=whiskerprops6a, flierprops=flierprops6a, boxprops=boxprops6a, capprops=capprops6a)
        plt.boxplot(ssp_div_d_df['Mean_Rel_Diff_%'].loc[(ssp_div_d_df['Year']==year)], vert=True, positions=[year+2.0], widths=0.6, 
                    medianprops=medianprops7a, whiskerprops=whiskerprops7a, flierprops=flierprops7a, boxprops=boxprops7a, capprops=capprops7a)
        plt.boxplot(ssp_div_d_df['Peak_Rel_Diff_%'].loc[(ssp_div_d_df['Year']==year)], vert=True, positions=[year+2.8], widths=0.6, 
                    medianprops=medianprops8a, whiskerprops=whiskerprops8a, flierprops=flierprops8a, boxprops=boxprops8a, capprops=capprops8a)
    plt.legend(loc='upper left', prop={'size': 12})
    plt.xlim([2015, 2095]); plt.xticks([2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090],['2020','2030','2040','2050','2060','2070','2080','2090'])
    plt.ylim(bottom=0)
    plt.grid(True)
    plt.ylabel('Annual Mean Hourly Load Difference [%]')
    plt.grid(True)
    plt.title('Distribution of Socioeconomic Scenario Divergence [SSP3 vs SSP5] Across States')
    plt.title('a)', loc='left', fontsize=13)
    
    # Plot the distribution of the change in annual total load across states:
    plt.subplot(312)
    plt.plot([0,0], color='k',      linestyle='-', label='Mean Load', linewidth=2)
    plt.plot([0,0], color='k',      linestyle=':', label='Peak Load', linewidth=2)
    plt.plot([0,0], color='forestgreen', linestyle='-', label=(rcp_div_a_one + ' vs ' + rcp_div_a_two), linewidth=2)
    plt.plot([0,0], color='blueviolet',  linestyle='-', label=(rcp_div_b_one + ' vs ' + rcp_div_b_two), linewidth=2)
    plt.plot([0,0], color='goldenrod',   linestyle='-', label=(rcp_div_c_one + ' vs ' + rcp_div_c_two), linewidth=2)
    plt.plot([0,0], color='lightcoral',  linestyle='-', label=(rcp_div_d_one + ' vs ' + rcp_div_d_two), linewidth=2)
    for year in range(2020,2100,10):
        plt.boxplot(rcp_div_a_df['Mean_Rel_Diff_%'].loc[(rcp_div_a_df['Year']==year)], vert=True, positions=[year-2.8], widths=0.6, 
                    medianprops=medianprops1b, whiskerprops=whiskerprops1b, flierprops=flierprops1b, boxprops=boxprops1b, capprops=capprops1b)
        plt.boxplot(rcp_div_a_df['Peak_Rel_Diff_%'].loc[(rcp_div_a_df['Year']==year)], vert=True, positions=[year-2.0], widths=0.6, 
                    medianprops=medianprops2b, whiskerprops=whiskerprops2b, flierprops=flierprops2b, boxprops=boxprops2b, capprops=capprops2b)
        plt.boxplot(rcp_div_b_df['Mean_Rel_Diff_%'].loc[(rcp_div_b_df['Year']==year)], vert=True, positions=[year-1.2], widths=0.6, 
                    medianprops=medianprops3b, whiskerprops=whiskerprops3b, flierprops=flierprops3b, boxprops=boxprops3b, capprops=capprops3b)
        plt.boxplot(rcp_div_b_df['Peak_Rel_Diff_%'].loc[(rcp_div_b_df['Year']==year)], vert=True, positions=[year-0.4], widths=0.6, 
                    medianprops=medianprops4b, whiskerprops=whiskerprops4b, flierprops=flierprops4b, boxprops=boxprops4b, capprops=capprops4b)
        plt.boxplot(rcp_div_c_df['Mean_Rel_Diff_%'].loc[(rcp_div_c_df['Year']==year)], vert=True, positions=[year+0.4], widths=0.6, 
                    medianprops=medianprops5b, whiskerprops=whiskerprops5b, flierprops=flierprops5b, boxprops=boxprops5b, capprops=capprops5b)
        plt.boxplot(rcp_div_c_df['Peak_Rel_Diff_%'].loc[(rcp_div_c_df['Year']==year)], vert=True, positions=[year+1.2], widths=0.6, 
                    medianprops=medianprops6b, whiskerprops=whiskerprops6b, flierprops=flierprops6b, boxprops=boxprops6b, capprops=capprops6b)
        plt.boxplot(rcp_div_d_df['Mean_Rel_Diff_%'].loc[(rcp_div_d_df['Year']==year)], vert=True, positions=[year+2.0], widths=0.6, 
                    medianprops=medianprops7b, whiskerprops=whiskerprops7b, flierprops=flierprops7b, boxprops=boxprops7b, capprops=capprops7b)
        plt.boxplot(rcp_div_d_df['Peak_Rel_Diff_%'].loc[(rcp_div_d_df['Year']==year)], vert=True, positions=[year+2.8], widths=0.6, 
                    medianprops=medianprops8b, whiskerprops=whiskerprops8b, flierprops=flierprops8b, boxprops=boxprops8b, capprops=capprops8b)
    plt.legend(loc='upper left', prop={'size': 12})
    plt.xlim([2015, 2095]); plt.xticks([2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090],['2020','2030','2040','2050','2060','2070','2080','2090'])
    plt.ylim(bottom=0)
    plt.grid(True)
    plt.ylabel('Annual Mean Hourly Load Difference [%]')
    plt.grid(True)
    plt.title('Distribution of Climate Scenario Divergence [RCP 4.5 vs RCP 8.5] Across States')
    plt.title('b)', loc='left', fontsize=13)   
    
    
    # Plot the distribution of the change in annual total load across states:
    plt.subplot(313)
    plt.plot([0,0], color='k',      linestyle='-',  label='Mean Load', linewidth=2)
    plt.plot([0,0], color='k',      linestyle=':',  label='Peak Load', linewidth=2)
    plt.plot([0,0], color='brown',   linestyle='-', label=(gcm_div_a_one + ' vs ' + gcm_div_a_two), linewidth=2)
    plt.plot([0,0], color='gold',    linestyle='-', label=(gcm_div_b_one + ' vs ' + gcm_div_b_two), linewidth=2)
    plt.plot([0,0], color='skyblue', linestyle='-', label=(gcm_div_c_one + ' vs ' + gcm_div_c_two), linewidth=2)
    plt.plot([0,0], color='magenta', linestyle='-', label=(gcm_div_d_one + ' vs ' + gcm_div_d_two), linewidth=2)
    for year in range(2020,2100,10):
        plt.boxplot(gcm_div_a_df['Mean_Rel_Diff_%'].loc[(gcm_div_a_df['Year']==year)], vert=True, positions=[year-2.8], widths=0.6, 
                    medianprops=medianprops1c, whiskerprops=whiskerprops1c, flierprops=flierprops1c, boxprops=boxprops1c, capprops=capprops1c)
        plt.boxplot(gcm_div_a_df['Peak_Rel_Diff_%'].loc[(gcm_div_a_df['Year']==year)], vert=True, positions=[year-2.0], widths=0.6, 
                    medianprops=medianprops2c, whiskerprops=whiskerprops2c, flierprops=flierprops2c, boxprops=boxprops2c, capprops=capprops2c)
        plt.boxplot(gcm_div_b_df['Mean_Rel_Diff_%'].loc[(gcm_div_b_df['Year']==year)], vert=True, positions=[year-1.2], widths=0.6, 
                    medianprops=medianprops3c, whiskerprops=whiskerprops3c, flierprops=flierprops3c, boxprops=boxprops3c, capprops=capprops3c)
        plt.boxplot(gcm_div_b_df['Peak_Rel_Diff_%'].loc[(gcm_div_b_df['Year']==year)], vert=True, positions=[year-0.4], widths=0.6, 
                    medianprops=medianprops4c, whiskerprops=whiskerprops4c, flierprops=flierprops4c, boxprops=boxprops4c, capprops=capprops4c)
        plt.boxplot(gcm_div_c_df['Mean_Rel_Diff_%'].loc[(gcm_div_c_df['Year']==year)], vert=True, positions=[year+0.4], widths=0.6, 
                    medianprops=medianprops5c, whiskerprops=whiskerprops5c, flierprops=flierprops5c, boxprops=boxprops5c, capprops=capprops5c)
        plt.boxplot(gcm_div_c_df['Peak_Rel_Diff_%'].loc[(gcm_div_c_df['Year']==year)], vert=True, positions=[year+1.2], widths=0.6, 
                    medianprops=medianprops6c, whiskerprops=whiskerprops6c, flierprops=flierprops6c, boxprops=boxprops6c, capprops=capprops6c)
        plt.boxplot(gcm_div_d_df['Mean_Rel_Diff_%'].loc[(gcm_div_d_df['Year']==year)], vert=True, positions=[year+2.0], widths=0.6, 
                    medianprops=medianprops7c, whiskerprops=whiskerprops7c, flierprops=flierprops7c, boxprops=boxprops7c, capprops=capprops7c)
        plt.boxplot(gcm_div_d_df['Peak_Rel_Diff_%'].loc[(gcm_div_d_df['Year']==year)], vert=True, positions=[year+2.8], widths=0.6, 
                    medianprops=medianprops8c, whiskerprops=whiskerprops8c, flierprops=flierprops8c, boxprops=boxprops8c, capprops=capprops8c)
    plt.legend(loc='upper left', prop={'size': 12})
    plt.xlim([2015, 2095]); plt.xticks([2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090],['2020','2030','2040','2050','2060','2070','2080','2090'])
    plt.ylim(bottom=0)
    plt.grid(True)
    plt.ylabel('Annual Mean Hourly Load Difference [%]')
    plt.grid(True)
    plt.title('Distribution of Climate Model Divergence [Hotter vs Colder Models] Across States')
    plt.title('c)', loc='left', fontsize=13)   
    
    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       filename = ('State_Load_Divergence_Distributions.png')
       plt.savefig(os.path.join(image_output_dir, filename), dpi=image_resolution, bbox_inches='tight', facecolor='white')
    

In [10]:
plot_state_difference_box_plots(data_input_dir = data_output_dir, 
                                image_output_dir = image_output_dir, 
                                image_resolution = 300, 
                                save_images = True)
